In [36]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [37]:
# load data
Data = pd.read_csv("Life Expectancy Data.csv")
cols = Data.columns
print(Data.columns)
print(Data)

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')
          Country  Year      Status  Life expectancy   Adult Mortality  \
0     Afghanistan  2015  Developing              65.0            263.0   
1     Afghanistan  2014  Developing              59.9            271.0   
2     Afghanistan  2013  Developing              59.9            268.0   
3     Afghanistan  2012  Developing              59.5            272.0   
4     Afghanistan  2011  Developing              59.2            275.0   
...           ...   ...         ...               ...              ...   
2933     Zimbabwe  2004  Developing              44.3         

In [38]:
# Find the percentage of None data for each colume
Data.isna().sum()/len(Data)

Country                            0.000000
Year                               0.000000
Status                             0.000000
Life expectancy                    0.003404
Adult Mortality                    0.003404
infant deaths                      0.000000
Alcohol                            0.066031
percentage expenditure             0.000000
Hepatitis B                        0.188223
Measles                            0.000000
 BMI                               0.011572
under-five deaths                  0.000000
Polio                              0.006467
Total expenditure                  0.076923
Diphtheria                         0.006467
 HIV/AIDS                          0.000000
GDP                                0.152485
Population                         0.221920
 thinness  1-19 years              0.011572
 thinness 5-9 years                0.011572
Income composition of resources    0.056841
Schooling                          0.055480
dtype: float64

In [39]:
# Drop rows that has no "live expectancy" column since it is just 0.003404
Data = Data[~Data["Life expectancy "].isna()]

# Drop rows that are missing values in cloumns with more than 95% of the values 
non_perc = (Data.isna().sum()/len(Data)).values
less5Cols = non_perc < 0.05
print(cols[less5Cols])
for col in cols[less5Cols]:
    if Data[col].isna().sum()/len(Data) > 1e-3:
        print(col)
        print("before :", len(Data))
        Data = Data[~Data[col].isna()]
        print("after :",len(Data))   

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'percentage expenditure', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Diphtheria ', ' HIV/AIDS',
       ' thinness  1-19 years', ' thinness 5-9 years'],
      dtype='object')
 BMI 
before : 2928
after : 2896
Polio
before : 2896
after : 2888


In [40]:
print()
print("After droping")
print(Data.isna().sum()/len(Data)) 


After droping
Country                            0.000000
Year                               0.000000
Status                             0.000000
Life expectancy                    0.000000
Adult Mortality                    0.000000
infant deaths                      0.000000
Alcohol                            0.060596
percentage expenditure             0.000000
Hepatitis B                        0.181787
Measles                            0.000000
 BMI                               0.000000
under-five deaths                  0.000000
Polio                              0.000000
Total expenditure                  0.073407
Diphtheria                         0.000000
 HIV/AIDS                          0.000000
GDP                                0.150623
Population                         0.222992
 thinness  1-19 years              0.000000
 thinness 5-9 years                0.000000
Income composition of resources    0.055402
Schooling                          0.055402
dtype: float64


In [41]:
# replace Developing , Developed in boolean
Data = Data.replace("Developing", False)
Data = Data.replace("Developed", True)
print(Data["Status"])

0       False
1       False
2       False
3       False
4       False
        ...  
2933    False
2934    False
2935    False
2936    False
2937    False
Name: Status, Length: 2888, dtype: bool


In [42]:
print(Data)

          Country  Year  Status  Life expectancy   Adult Mortality  \
0     Afghanistan  2015   False              65.0            263.0   
1     Afghanistan  2014   False              59.9            271.0   
2     Afghanistan  2013   False              59.9            268.0   
3     Afghanistan  2012   False              59.5            272.0   
4     Afghanistan  2011   False              59.2            275.0   
...           ...   ...     ...               ...              ...   
2933     Zimbabwe  2004   False              44.3            723.0   
2934     Zimbabwe  2003   False              44.5            715.0   
2935     Zimbabwe  2002   False              44.8             73.0   
2936     Zimbabwe  2001   False              45.3            686.0   
2937     Zimbabwe  2000   False              46.0            665.0   

      infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   \
0                62     0.01               71.279624         65.0      1154   
1

In [43]:
thing = Data.groupby("Country").mean()
# for col in thing.columns:
#     thing.fillna({col : thing[col].mean()}, inplace=True)
# print(col["Algeria"])
# print(thing)
# print(thing["Schooling"][-10:])
print(thing["Schooling"][-10:])

Country
United Republic of Tanzania                NaN
United States of America                   NaN
Uruguay                               15.23125
Uzbekistan                            11.64375
Vanuatu                               10.56875
Venezuela (Bolivarian Republic of)    12.78750
Viet Nam                              11.51250
Yemen                                  8.50625
Zambia                                11.21250
Zimbabwe                               9.82500
Name: Schooling, dtype: float64


In [44]:
# fill missing values in each column with the mean of the country's values in that column
Data = Data.reset_index()
Data.drop('index', axis=1, inplace=True)
print(Data.columns)
non_perc = (Data.isna().sum()/len(Data)).values
ColsWithNone = non_perc > 0
print(cols[ColsWithNone])
new_Data = Data.copy()
country_avgs = Data.groupby("Country").mean()
for col in country_avgs.columns:
    country_avgs.fillna({col : country_avgs[col].mean()}, inplace=True)
for col in cols[ColsWithNone]:
    avgs = country_avgs[col]
    indx = Data[col].isna()
    for i in np.where(indx)[0]:
        print(new_Data.loc[i,col])
        new_Data.loc[i,col] = avgs[Data["Country"][i]]
        print(col)
        print(Data["Country"][i])
        print(new_Data.loc[i,col])

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')
Index(['Alcohol', 'Hepatitis B', 'Total expenditure', 'GDP', 'Population',
       'Income composition of resources', 'Schooling'],
      dtype='object')
nan
Alcohol
Algeria
0.4066666666666667
nan
Alcohol
Angola
5.740666666666667
nan
Alcohol
Antigua and Barbuda
7.949333333333333
nan
Alcohol
Argentina
7.966666666666667
nan
Alcohol
Armenia
3.7026666666666666
nan
Alcohol
Australia
10.155333333333335
nan
Alcohol
Austria
12.235999999999999
nan
Alcohol
Azerbaijan
0.9919999999999999
nan
Alcohol
Bahamas
10.271333333333333
nan
Alcohol
Bahrain
1.9346666666666665
nan
Alcohol
Bangladesh

Alcohol
Saint Lucia
11.511333333333333
nan
Alcohol
Saint Vincent and the Grenadines
6.11
nan
Alcohol
Samoa
2.704
nan
Alcohol
Sao Tome and Principe
4.191333333333334
nan
Alcohol
Saudi Arabia
0.07466666666666667
nan
Alcohol
Senegal
0.30933333333333335
nan
Alcohol
Serbia
8.682666666666668
nan
Alcohol
Seychelles
6.201333333333333
nan
Alcohol
Sierra Leone
3.1546666666666665
nan
Alcohol
Slovakia
10.473333333333333
nan
Alcohol
Slovenia
10.754666666666667
nan
Alcohol
Solomon Islands
0.8006666666666666
nan
Alcohol
Somalia
0.01
nan
Alcohol
South Africa
7.588666666666667
nan
Alcohol
Spain
10.024
nan
Alcohol
Sri Lanka
2.096
nan
Alcohol
Suriname
4.984
nan
Alcohol
Swaziland
4.463333333333334
nan
Alcohol
Sweden
6.926666666666667
nan
Alcohol
Switzerland
10.338
nan
Alcohol
Syrian Arab Republic
0.8046666666666666
nan
Alcohol
Tajikistan
0.33066666666666666
nan
Alcohol
Thailand
6.131333333333333
nan
Alcohol
The former Yugoslav republic of Macedonia
1.8586666666666667
nan
Alcohol
Timor-Leste
0.194615384615

In [45]:
# save the cleaned data
new_Data.to_csv("Life Expectancy Data Cleaned.csv")